## 0. PSEUDOCODE / OVERVIEW

##### Prep data
Merge countries' hazard impact vectors into one gdf.

##### Spatial join to project ADMs
Get centroids of hazard gdf.
<br> For each ADM, spatial join ADM with centroids (contains).

##### Combine into dataframe
Provide unique field names.
<br> Table join on ADM3 code.

## 1. PREPARE WORKSPACE

### 1.1 Load all packages.

In [ ]:
# Built-in:
# dir(), print(), range(), format(), int(), len(), list(), max(), min(), zip(), sorted(), sum(), open(), del, = None, try except, with as, for in, if elif else
# Also: list.append(), list.insert(), list.remove(), count(), startswith(), endswith(), contains(), replace()

import os, sys, glob, re, time, subprocess, string # os.getcwd(), os.path.join(), os.listdir(), os.remove(), time.ctime(), glob.glob(), string.zfill(), string.join()
from os.path import exists # exists()
from functools import reduce # reduce()

import geopandas as gpd # read_file(), GeoDataFrame(), sjoin_nearest(), to_crs(), to_file(), .crs, buffer(), dissolve()
import pandas as pd # .dtypes, Series(), concat(), DataFrame(), read_table(), merge(), to_csv(), .loc[], head(), sample(), astype(), unique(), rename(), between(), drop(), fillna(), idxmax(), isna(), isin(), apply(), info(), sort_values(), notna(), groupby(), value_counts(), duplicated(), drop_duplicates()
from shapely.geometry import Point, LineString, Polygon, shape, MultiPoint
from shapely.ops import cascaded_union
from shapely.validation import make_valid  # in apply(make_valid)
import shapely.wkt

import numpy as np # median(), mean(), tolist(), .inf
import fiona, rioxarray # fiona.open()
import rasterio # open(), write_band(), .name, .count, .width, .height. nodatavals, .meta, update(), copy(), write()
from rasterio.plot import show
from rasterio import features # features.rasterize()
from rasterio.features import shapes
from rasterio import mask # rasterio.mask.mask()
from rasterio.enums import Resampling # rasterio.enums.Resampling()
from rasterstats import zonal_stats
from osgeo import gdal, osr, ogr, gdal_array, gdalconst # Open(), SpatialReference, WarpOptions(), Warp(), GetDataTypeName(), GetRasterBand(), GetNoDataValue(), Translate(), GetProjection(), GetAttrValue()

In [ ]:
# The usual directories
Project_Fd = os.getcwd()
Current_Fd = os.path.join(Project_Fd, 'Hazard')
Source_Fd = os.path.join(Current_Fd, 'Source', '2022')
Intermed_Fd = os.path.join(Current_Fd, 'Intermediate')

# Auxilliary sources
ADM_Fd = os.path.join(Project_Fd, 'ADM')
ADM_gpkg = os.path.join(ADM_Fd, 'Sahel_AdminBoundaries.gpkg')
CCDR_Fd = 'Q:\GIS\povertyequity\CCDR'

# Check paths
print('\n\n'.join([Project_Fd, Current_Fd, Source_Fd, Intermed_Fd, CCDR_Fd, ADM_Fd, ADM_gpkg]))

## 2. PREP ADMIN LAYERS

### 2.1 CCDR-to-PTI dictionary

Note: this dictionary was made manually in QGIS based on admin boundary alignments between the incongruent admin area datasets.

CCDR admin area count: (remember that one country is missing in each admin set)
<br>ADM2: 182
<br>ADM3: 1318

PTI admin area count:
<br>ADM2: 275
<br>ADM3: 1433

#### CCDR-PTI dictionary

In [ ]:
CCDR_ADM2 = pd.DataFrame(gpd.read_file(
    os.path.join(Intermed_Fd, 'CCDR_PTI_dict.gpkg'), layer='ADM2'))

CCDR_ADM3 = pd.DataFrame(gpd.read_file(
    os.path.join(Intermed_Fd, 'CCDR_PTI_dict.gpkg'), layer='ADM3')).drop(columns=['fid', 'CCDR'])

print(CCDR_ADM2.info(), CCDR_ADM3.info(), '\n\n', CCDR_ADM2.sample(10), '\n\n', CCDR_ADM3.sample(10))

#### Pivot longer: ADM2

In [ ]:
Dict_ADM2 = pd.melt(CCDR_ADM2, id_vars=['CCDR_ID'], value_vars=[col for col in CCDR_ADM2 if col.startswith('ADM2')], 
                    value_name='PTI_ID')
Dict_ADM2 = Dict_ADM2.loc[Dict_ADM2['PTI_ID'].notnull()][['CCDR_ID', 'PTI_ID']]
Dict_ADM2

In [ ]:
print('Number of PTI polygons associated with more than one CCDR polygon: ', 
      (len(Dict_ADM2.index.unique()) - len(Dict_ADM2['CCDR_ID'].unique())))
print('Number of CCDR polygons associated with more than one PTI polygon: ', 
      (len(Dict_ADM2.index.unique()) - len(Dict_ADM2['PTI_ID'].unique())))

#### Pivot longer: ADM3

In [ ]:
Dict_ADM3 = pd.melt(CCDR_ADM3, id_vars=['CCDR_ID'], value_vars=[col for col in CCDR_ADM3 if col.startswith('ADM3')], 
                    value_name='PTI_ID')
Dict_ADM3 = Dict_ADM3.loc[Dict_ADM3['PTI_ID'].notnull()][['CCDR_ID', 'PTI_ID']]
Dict_ADM3

In [ ]:
print('Number of PTI polygons associated with more than one CCDR polygon: ', 
      (len(Dict_ADM3.index.unique()) - len(Dict_ADM3['CCDR_ID'].unique())))
print('Number of CCDR polygons associated with more than one PTI polygon: ', 
      (len(Dict_ADM3.index.unique()) - len(Dict_ADM3['PTI_ID'].unique())))

## 3. MATCH RISK INDICATORS WITH PTI ADMIN AREAS

### 3.1 Clean up risk layers

#### Load all indicators as separate ADM2 and ADM3 objects.

In [ ]:
Indic_ADM2 = pd.DataFrame(gpd.read_file(os.path.join(Intermed_Fd, 'CCDR_indicators.gpkg'), 
                                        layer='CCDR_ADM2')).drop(columns=['geometry', 'CCDR'])
Indic_ADM3 = pd.DataFrame(gpd.read_file(os.path.join(Intermed_Fd, 'CCDR_indicators.gpkg'), 
                                        layer='CCDR_ADM3')).drop(columns=['geometry', 'ADM2_CODE'])
print(Indic_ADM2.info(), Indic_ADM3.info())

In [ ]:
print(Indic_ADM2.nunique(axis=0), '\n\n', Indic_ADM3.nunique(axis=0))

### 3.2 Categorize indicators by eligible aggregation or disaggregation method.

If splitting a hazard ADM into 2+ project ADMs, AND if indicator does not depend on the polygon size or amount (intensive):
<br> Indicator unchanged, applied to each ADM.

If splitting a hazard ADM into 2+ project ADMs, AND if indicator is dependent on the polygon size or amount (extensive):
<br> No data applied to each ADM.

If dissolving 2+ hazard ADMs into a project ADM, AND if indicator is intensive:
<br> No data applied to each ADM.

If dissolving 2+ hazard ADMs into a project ADM, AND if indicator is extensive:
<br> -- If indicator is count, apply sum.
<br> -- If indicator is average, apply no data.
<br> -- If indicator is max, apply max.

Drought percent: INTENSIVE, AVERAGE
- If >1 CCDR to a PTI, no data.
- If >1 PTI to a CCDR, orig value.

Landslide: EXTENSIVE, COUNT
- If >1 CCDR to a PTI, sum.
- If >1 PTI to a CCDR, no data. 

Flood: EXTENSIVE, COUNT
- If >1 CCDR to a PTI, sum.
- If >1 PTI to a CCDR, no data.

Flood percent: INTENSIVE, PROPORTION
- If >1 CCDR to a PTI, no data.
- If >1 PTI to a CCDR, orig value.

Heat stress: EXTENSIVE, COUNT
- If >1 CCDR to a PTI, sum.
- If >1 PTI to a CCDR, no data.

Heat stress percent: INTENSIVE, PROPORTION
- If >1 CCDR to a PTI, no data.
- If >1 PTI to a CCDR, orig value.

In [ ]:
print(Indic_ADM2.info(), '\n', Indic_ADM3.info())

In [ ]:
# Take the Indicators list and assign intensive or extensive.
Intensive = ['DR_S1_30', 'DR_S1_50', # DR
              'HS_popEAEp', # HS
              'FL_buEAIp', 'FL_popEAIp' # FL
              # LS
            ]

Extensive = [# DR
              'HS_pop', 'HS_popEAE', # HS
              'FL_ag', 'FL_agEAE', 'FL_bu', 'FL_buEAI', 'FL_pop', 'FL_popEAI', # FL
              'LS_bu', 'LS_buEXP', 'LS_pop', 'LS_popEXP' # LS
             ]

### 3.3 Group-by aggregations

#### Table join risk indicators onto project ADMs.

In [ ]:
ADM2 = Dict_ADM2.merge(Indic_ADM2, on='CCDR_ID', how='left')
ADM3 = Dict_ADM3.merge(Indic_ADM3, on='CCDR_ID', how='left')
print(ADM2.info(), ADM3.info())

#### Identify duplicates and mark them in the expanded dataset.

In [ ]:
# subset rows that have duplicate PTI ADM: dupePTI
dupePTI_A2 = ADM2[ADM2.duplicated(['PTI_ID'], keep='first')]
dupePTI_A2 = list(dupePTI_A2['PTI_ID'])

dupePTI_A3 = ADM3[ADM3.duplicated(['PTI_ID'], keep='first')]
dupePTI_A3 = list(dupePTI_A3['PTI_ID'])
dupePTI_A3

In [ ]:
# subset rows that have duplicate CCDR ADM: dupeCCDR
dupeCCDR_A2 = ADM2[ADM2.duplicated(['CCDR_ID'], keep='first')]
dupeCCDR_A2 = list(dupeCCDR_A2['CCDR_ID'])

dupeCCDR_A3 = ADM3[ADM3.duplicated(['CCDR_ID'], keep='first')]
dupeCCDR_A3 = list(dupeCCDR_A3['CCDR_ID'])
dupeCCDR_A3

In [ ]:
ADM2['dupeCCDR'] = 0 # Default value will be zero
ADM2['dupePTI'] = 0
ADM2.loc[ADM2.CCDR_ID.isin(dupeCCDR_A2), 'dupeCCDR'] = 1
ADM2.loc[ADM2.PTI_ID.isin(dupePTI_A2), 'dupePTI'] = 1

ADM3['dupeCCDR'] = 0
ADM3['dupePTI'] = 0
ADM3.loc[ADM3.CCDR_ID.isin(dupeCCDR_A3), 'dupeCCDR'] = 1
ADM3.loc[ADM3.PTI_ID.isin(dupePTI_A3), 'dupePTI'] = 1

print(ADM2.sample(10), ADM3.sample(10))

In [ ]:
# Check to make sure there are 2 unique values for the dupe fields.
print(ADM2.nunique(axis=0), ADM3.nunique(axis=0))

In [ ]:
# We can drop CCDR ADM field now
ADM2 = ADM2.drop(columns='CCDR_ID', axis=1)
ADM3 = ADM3.drop(columns='CCDR_ID', axis=1)

#### Group-by PTI ADMs and apply appropriate aggregation or null setting

In [ ]:
# Now we can work with just the PTI ADM features.
ADM2_Int = ADM2.drop(columns=Extensive).groupby(
    'PTI_ID', as_index=False).first() # Keep original value

ADM2_Ext = ADM2.drop(columns=Intensive).groupby(
    'PTI_ID', as_index=False).sum() # Sum these counts
ADM2_Int.info()

In [ ]:
ADM3_Int = ADM3.drop(columns=Extensive).groupby(
    'PTI_ID', as_index=False).first() # Keep original value

ADM3_Ext = ADM3.drop(columns=Intensive).groupby(
    'PTI_ID', as_index=False).sum() # Sum these counts
ADM3_Int.info()

In [ ]:
# Change extensive variables to None if there were multiple PTI ADMs
for col in Extensive:
    ADM2_Ext.loc[ADM2_Ext.dupePTI > 0, col] = None
# Change intensive variables to None if there were multiple CCDR ADMs
for col in Intensive:
    ADM2_Int.loc[ADM2_Int.dupeCCDR > 0, col] = None

In [ ]:
for col in Extensive:
    ADM3_Ext.loc[ADM3_Ext.dupePTI > 0, col] = None
for col in Intensive:
    ADM3_Int.loc[ADM3_Int.dupeCCDR > 0, col] = None

In [ ]:
# Merge intensive and extensive variables into single df and save to file.
ADM2_PTI = ADM2_Int.merge(ADM2_Ext, on='PTI_ID', how='outer')
ADM2_PTI = ADM2_PTI.loc[:, ~ADM2_PTI.columns.str.contains('dupe')]
ADM3_PTI = ADM3_Int.merge(ADM3_Ext, on='PTI_ID', how='outer')
ADM3_PTI = ADM3_PTI.loc[:, ~ADM3_PTI.columns.str.contains('dupe')]
print(ADM2_PTI.info(), ADM3_PTI.info())

In [ ]:
ADM2_PTI.sample(20)

In [ ]:
ADM3_PTI.sample(20)

### 3.4 Join onto parent PTI ADM dataframes and save to file.

In [ ]:
ADM2 = pd.DataFrame(gpd.read_file(filename=os.path.join(ADM_Fd, 'Sahel_AdminBoundaries.gpkg'), 
                     layer='adm2'))[['ADM1_CODE', 'ADM2_CODE']]
ADM3 = pd.DataFrame(gpd.read_file(filename=os.path.join(ADM_Fd, 'Sahel_AdminBoundaries.gpkg'), 
                     layer='adm3'))[['ADM1_CODE', 'ADM2_CODE', 'ADM3_CODE']]
print(ADM2.info(), ADM3.info())

In [ ]:
ADM2 = ADM2.merge(ADM2_PTI, left_on='ADM2_CODE', right_on='PTI_ID', how='left')
ADM3 = ADM3.merge(ADM3_PTI, left_on='ADM3_CODE', right_on='PTI_ID', how='left')
ADM2.to_csv(os.path.join(Current_Fd, 'Results', 'Risk_Temp_ADM2.csv'))
ADM3.to_csv(os.path.join(Current_Fd, 'Results', 'Risk_Temp_ADM3.csv'))

In [ ]:
Indic_ADM2.tail(30)

In [ ]:
ADM2.tail(30)